# 프로젝트 개요
- 강의명 : 2022년 K-디지털 직업훈련(Training) 사업 - AI데이터플랫폼을 활용한 빅데이터 분석전문가 과정
- 교과목명 : 빅데이터 분석 및 시각화, AI개발 기초, 인공지능 프로그래밍
- 프로젝트 주제 : Spaceship Titanic 데이터를 활용한 탑승유무 분류모형 개발
- 프로젝트 마감일 : 2022년 4월 12일 화요일


In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# step 1. 라이브러리 및 데이터 불러오기
- 라이브러리를 불러온다.

In [ ]:
# preprocessing
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)

In [ ]:
# visualization
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Missing Value processing
from sklearn.impute import SimpleImputer

In [ ]:
# Encoding
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
# Split
from sklearn.model_selection import train_test_split

In [ ]:
# Models
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Cross
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
import time

## 데이터 불러오기
- 훈련 데이터는 8,693개, 테스트 데이터는 4,277개.

In [ ]:
train_data = pd.read_csv("../input/spaceship-titanic/train.csv")
test_data = pd.read_csv("../input/spaceship-titanic/test.csv")
sample_submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")



print(train_data.shape)
print(test_data.shape)
print(sample_submission.shape)

## Column 설명
- PassengerId는 "AAAA_BB"형태이다. AAAA는 그룹을 나타내고, BB는 그룹안에서의 개개인을 지칭한다.
- HomePlanet은 고향별을 나타낸다.
- CryoSleep은 여행중에 냉동수면 여부를 나타낸다.
- cabin은 승객이 머무르는 공간을 나타낸다. deck/number/side 형태로 구성된다. side의 P는 Port를 나타내고, S는 Starboard를 나타낸다.
- Age는 승객의 나이이다.
- VIP는 승객이 special VIP service 비용을 지불했는지를 보여준다.
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck는 각각 우주선의 편의시설에 승객이 비용을 지불했는지를 나타낸다.
- Name은 승객의 이름으로, first name, last name 순이다.
- Transported 칼럼이 타깃 데이터이다. 승객이 다른 차원으로 운송되었는지 여부를 보여준다. 훈련 데이터에는 Transported 칼럼이 있고, 테스트 데이터에는 Transported 칼럼이 있다. Transported 칼럼을 예측해야 할 것이다.

## train data
- 훈련 데이터를 살펴본다.
- 훈련 데이터에는 14개의 열이 있고, 8,693개의 행(샘플)이 있다.
- 훈련 데이터에는 119,378개의 특성이 있다.
- 훈련 데이터안에는 2,324개의 결측치가 있다.
- Transported 열은 훈련 데이터 안에서만 사용할 수 있다.
- 14개의 열 중에서 CryoSleep 열 안에 217개로 가장 많은 결측치가 있다. 

In [ ]:
# first 5 rows of train dataset
train_data.head()

In [ ]:
train_data.info()

In [ ]:

print(f'\033[94mNumber of rows in train data: {train_data.shape[0]}')
print(f'\033[95mNumber of columns in train data: {train_data.shape[1]}')
print(f'\033[92mNumber of vlaues in train data: {train_data.count().sum()}')
print(f'\033[93mNumber of with missing values in train date: {sum(train_data.isna().sum())}')

In [ ]:
print(train_data.isna().sum().sort_values(ascending = False))

- 갯수, 평균, 표준편차, 최솟값, 최댓값, 4분위수 등의 기초 통계량을 확인한다.

In [ ]:
train_data.describe()

## test data
- 테스트 데이터에는 13개의 열이 있고, 4,277개의 행(샘플)이 있다.
- 테스트 데이터에는 54,484개의 특성이 있다.
- 훈련 데이터안에는 1,117개의 결측치가 있다.
- 13개의 열 중에서 FoodCourt 열 안에 217개로 가장 많은 결측치가 있다.
- 종속변수는 Transported임을 알 수 있다.

In [ ]:
# first 5 rows of test data 
test_data.head()


In [ ]:
test_data.info()

In [ ]:
print(f'\033[94mNumber of rows in test data: {test_data.shape[0]}')
print(f'\033[95mNumber of columns in test data: {test_data.shape[1]}')
print(f'\033[92mNumber of vlaues in test data: {test_data.count().sum()}')
print(f'\033[93mNumber of rows with missing values in test date: {sum(test_data.isna().sum())}')

In [ ]:
print((test_data.isna().sum().sort_values(ascending = False)))

- 갯수, 평균, 표준편차, 최솟값, 최댓값, 4분위수 등의 기초 통계량을 확인한다.

In [ ]:
test_data.describe()

### Sample_Submission File
- 식별할 수 있는 PassengerId 열과 종속변수인 Transported 열로 구성되어있다.

In [ ]:
# first 5 rows of submission file
sample_submission.head()

In [ ]:
sample_submission.info()

# Step 2. 탐색적 자료 분석(EDA)

## 데이터 시각화를 위한 주요 함수
- 우선 종속변수의 갯수에 대해 확인한다.
- boolean값이 비슷하게 분포 되어 있다.

In [ ]:
train_data['Transported'].value_counts()

In [ ]:
train_data['CryoSleep'].value_counts()

- 시각화를 위한 함수를 정의한다.

In [ ]:
def cnt_bar(data, col_name):
    df = data[col_name].value_counts()
    fig, ax = plt.subplots(figsize=(10, 8))
    labels = [str(item) for item in list(data[col_name].value_counts().index)]
    bars = sns.countplot(x=col_name, data=data, color='lightgray', alpha=0.9, zorder=2, ax=ax)
    
    for bar in bars.patches:
        fontweight = "normal"
        color = "k"
        height = np.round(bar.get_height(), 2)
        if bar.get_height() == data[col_name].value_counts().values[0]:
            fontweight="bold"
            color="hotpink"
            bar.set_facecolor(color)
        ax.text(bar.get_x() + bar.get_width()/2., height+100, height+1, ha = 'center', size=12, fontweight=fontweight, color=color)
    ax.set_title(f'Bar Graph of {col_name}', size = 20)
    ax.set_xlabel(col_name, size = 20)
    ax.set_ylabel("No. Passengers", size = 20)

    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_position(("outward", 20))
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.grid(axis="y", which="major", color="lightgray")
    ax.grid(axis="y", which="minor", ls=":")
    
    plt.show()

- 함수를 적용해본다.
    - Cabin, PassengerId 열은 적용할 수 없다.

In [ ]:
cnt_bar(train_data, "Destination")

In [ ]:
cnt_bar(train_data, "HomePlanet")

In [ ]:
cnt_bar(train_data, "CryoSleep")

In [ ]:
cnt_bar(train_data, "VIP")

In [ ]:
cnt_bar(train_data, "Transported")

- RoomService, FoodCourt, Spa는 연속형 데이터가 아니라 비연속형 데이터임을 확인한다.

In [ ]:
train_data.describe()

In [ ]:
numeric_features = train_data.select_dtypes(include=[np.number])
numeric_features.columns

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(16, 10)) # 그래프의 행과 열 지정 및, 이미지 사이즈 지정
data = train_data.copy()
# data[numeric_features.columns].columns[0:]
for i, col in enumerate(data[numeric_features.columns].columns[0:]): # 좌표 평면 지정
    # print(i, col)
    if i <= 2:
        sns.boxplot(x=data["Transported"], y=data[col], ax=ax[0,i]) # 1행 좌표 평면
    else: 
        sns.boxplot(x=data["Transported"], y=data[col], ax=ax[1,i-4]) # 5행 좌표 평면
fig.suptitle('My Box Plots')
fig.tight_layout()
fig.subplots_adjust(top=0.95)

In [ ]:
train_data['RoomService'].value_counts()

In [ ]:
train_data['FoodCourt'].value_counts()

In [ ]:
train_data['Spa'].value_counts()

# Step 3. 데이터 전처리


## 불필요한 변수 제거

- PassengerId, Name, Cabin 열은 쓰지 않는다.
- 따라서, 제거한다.

In [ ]:
train_data['Name'].value_counts()

In [ ]:
train_data['Cabin'].value_counts()

In [ ]:
remove_cols = ['PassengerId', 'Name', 'Cabin']
passenferId = test_data['PassengerId']

print("Before:", train_data.shape, test_data.shape)
train_data = train_data.drop(remove_cols, axis = 1)
test_data = test_data.drop(remove_cols, axis= 1)

print("After:", train_data.shape, test_data.shape)

## 결측치
- 결측치 데이터를 확인한다.
- 결측치 데이터를 채워 넣는다.
- 우선, 수치형 데이터부터 해준다.

In [ ]:
train_data.isnull().sum().sort_values(ascending = False)

In [ ]:
test_data.isnull().sum().sort_values(ascending = False)

In [ ]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
STRATEGY = 'median'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train_data[imputer_cols])
train_data[imputer_cols] = imputer.transform(train_data[imputer_cols])
test_data[imputer_cols] = imputer.transform(test_data[imputer_cols])

print("train:\n", train_data.isnull().sum())
print("----------------")
print("test:\n", test_data.isnull().sum())

- 문자열 데이터를 처리한다.

In [ ]:
imputer_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]
STRATEGY = 'most_frequent'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train_data[imputer_cols])
train_data[imputer_cols] = imputer.transform(train_data[imputer_cols])
test_data[imputer_cols] = imputer.transform(test_data[imputer_cols])

print("train:\n", train_data.isnull().sum())
print("---")
print("test:\n", test_data.isnull().sum())

## Categorical Feature Encoding
- 문자열의 경우 수치형으로 변환해주어야 한다.
- OneHot Encoding 방식

### Onehot Encoding


In [ ]:
train_data['Transported'] = train_data['Transported'].map({True: 1, False: 0})
categorical_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

transformer = make_column_transformer(
    (OneHotEncoder(), categorical_cols),
    remainder='passthrough')

train_transformed = transformer.fit_transform(train_data[categorical_cols])
train_transformed_df = pd.DataFrame(train_transformed, columns=transformer.get_feature_names_out())
train_data = pd.concat([train_data, train_transformed_df], axis = 1)
train_data = train_data.drop(categorical_cols, axis = 1)

test_transformed = transformer.fit_transform(test_data[categorical_cols])
test_transformed_df = pd.DataFrame(test_transformed, columns=transformer.get_feature_names_out())
test_data = pd.concat([test_data, test_transformed_df], axis = 1)
test_data = test_data.drop(categorical_cols, axis = 1)

- 종속변수를 제외한 모든 열이 실수형이 된 것을 확인한다.

In [ ]:
train_data.info()

In [ ]:
test_data.info()

# Step 4. 머신러닝 모형 개발


- 독립변수와 족송변수를 구분한다.

In [ ]:
X_cols = test_data.columns
X = train_data[X_cols].to_numpy()
y = train_data['Transported'].to_numpy()

- 훈련 데이터에서 일부를 검증데이터로 떼어낸다. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

- LightGBM 클래스를 임포트한다.

In [ ]:
lgb = LGBMClassifier(random_state=42)
lgb

## Model Selection

- Logistic Regression는 선형 방정식을 사용한 분류 모형이다. 시그모이드 함수나 소프트맥스 함수를 사용하여 확률을 알 수 있다.
- K-NeighborsClassifier 는 맨해튼 거리 또는 유클라디안 거리를 사용해서 이웃의 개수가 많은 쪽으로 데이터를 분류하는 기법이다.
- RandomForest는 대표적인 의사결정트리 모형의 앙상블 학습 방법이다. 보통 회귀나 분류 문제를 위해 사용된다. 부트스트랩 샘플을 사용하고 랜덤하게 특성을 선택하여 트리를 만드는 방식을 사용한다.
- Light Gradient Boosting Machine(LGBM)은 XGBoost와 기본적으로 같고, 성능도 비슷하나, 더욱 빠르므로 선택했다. 참고로, XGBoost는 의사결정트리 모형의 앙상블로 구성된다.


- 모형을 정의한다.

In [ ]:
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

# Classifiers
classifiers = {
    "LogisticRegression" : LogisticRegression(random_state=0),
    "KNN" : KNeighborsClassifier(),
    "RandomForest" : RandomForestClassifier(random_state=0),
    "LGBM" : LGBMClassifier(random_state=0)
}

- 매개 변수를 바꿔가며 모델을 훈련 시킬 것이다.
- 그리드 서치를 위한 옵션을 선택한다.

In [ ]:
# Grids for grid search
LR_grid = {'penalty': ['l1','l2'],
           'C': [0.25, 0.5, 0.75, 1, 1.25, 1.5],
           'max_iter': [50, 100, 150]}

KNN_grid = {'n_neighbors': [3, 5, 7, 9],
            'p': [1, 2]}

RF_grid = {'n_estimators': [50, 100, 150, 200, 250, 300],
        'max_depth': [4, 6, 8, 10, 12]}

boosted_grid = {'n_estimators': [50, 100, 150, 200],
        'max_depth': [4, 8, 12],
        'learning_rate': [0.05, 0.1, 0.15]}

# Dictionary of all grids
grid = {
    "LogisticRegression" : LR_grid,
    "KNN" : KNN_grid,
    "RandomForest" : RF_grid,
    "LGBM" : boosted_grid
}

- 모형과 그리드 서치를 정의한다.

In [ ]:
# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
import time

i=0
clf_best_params=classifiers.copy()
valid_scores=pd.DataFrame({'Classifer':classifiers.keys(), 'Validation accuracy': np.zeros(len(classifiers)), 'Training time': np.zeros(len(classifiers))})
for key, classifier in classifiers.items():
    start = time.time()
    clf = GridSearchCV(estimator=classifier, param_grid=grid[key], n_jobs=-1, cv=None)

    # Train and score
    clf.fit(X_train, y_train)
    valid_scores.iloc[i,1]=clf.score(X_val, y_val)

    # Save trained model
    clf_best_params[key]=clf.best_params_
    
    # Print iteration and training time
    stop = time.time()
    valid_scores.iloc[i,2]=np.round((stop - start)/60, 2)
    
    print('Model:', key)
    print('Training time (mins):', valid_scores.iloc[i,2])
    print('')
    i+=1

- 4개의 모형중에서 Light Gradient Boosting Machine(LGBM)이 훈련 속도도 빨랐고, 정확도도 0.792561로 가장 높았다.
- 그 다음, RandomForest, K-Nearest Neighbors(KNN), LogisticRegression 순이다.

In [ ]:
# Show results
valid_scores

- 가장 좋은 파라미터를 찾는다.
- LogisticRegression 같은 경우, 규제를 제어하는 매개변수 C값은 1(기본값), 반복횟수 150회, 계수의 제곱을 규제하는 L2규제일 때 성능이 가장 좋았다.
- KNN 같은 경우, 근처 값들이 9개, 유클라디안 거리를 활용할 때 성능이 가장 좋았다.
- RandomForest 같은 경우, 깊이 10, 트리의 개수가 50일때, 성능이 가장 좋았다.
- LGBM 같은 경우, 학습속도 0.1, 깊이 8, 트리의 개수가 50일때, 성능이 가장 좋았다. 

In [ ]:
clf_best_params

- 최고의 파라미터를 모델에 적용한다.

In [ ]:
# Classifiers
best_classifiers = {
    "RandomForest" : RandomForestClassifier(**clf_best_params["RandomForest"], random_state=0),
    "LGBM" : LGBMClassifier(**clf_best_params["LGBM"], random_state=0)
}

- 10-폴드 교차 검증을 수행한다.

In [ ]:
# Number of folds in cross validation
FOLDS=10

preds=np.zeros(len(test_data))
for key, classifier in best_classifiers.items():
    start = time.time()
    
    # 5-fold cross validation
    cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)
    
    score=0
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        # Get training and validation sets
        X_train, X_valid = X[train_idx], X[val_idx]
        y_train, y_valid = y[train_idx], y[val_idx]

        # Train model
        clf = classifier
        clf.fit(X_train, y_train)

        # Make predictions and measure accuracy
        preds += clf.predict_proba(test_data)[:,1]
        score += clf.score(X_valid, y_valid)

    # Average accuracy    
    score=score/FOLDS
    
    # Stop timer
    stop = time.time()

    # Print accuracy and time
    print('Model:', key)
    print('Average validation accuracy:', np.round(100*score,2))
    print('Training time (mins):', np.round((stop - start)/60,2))
    print('')
    
# Ensemble predictions
preds=preds/(FOLDS*len(best_classifiers))

# Step 5. 모형 평가


- 가장 성능이 좋았던 lgbm 모델을 교차 검증한다.
- StratifiedKFold를 사용하여 훈련 세트를 섞어주고 5-폴드 교차 검증한다.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

splitter = StratifiedKFold(n_splits = 5, shuffle = True, random_state=42)
scores = cross_validate(lgb, X_train, y_train, return_train_score = True, cv=splitter)

print("train Acc.", np.mean(scores['train_score']))
print("test Acc.", np.mean(scores['test_score']))

- 검증 데이터를 활용하여 정확도를 예상한다.

In [ ]:
from sklearn.metrics import accuracy_score

lgb.fit(X_train, y_train)
y_pred = lgb.predict(X_val)
print("Acc.", accuracy_score(y_val, y_pred))

## 혼동행렬(Coufusion Matrix)
- 정확도(Accuracy)는 True를 True라고 옳게 예측한 경우와, False를 False라고 옳게 예측한 것의 비율이다.
- 어떤 데이터를 예측할 때, 네 가지 상황이 있을 수 있다.
- TP(True Positive) : True를 True라고 올바르게 맞다고 예측한 경우.
- TN(True Negative) : False를 False라고 예측한 경우.
- FP(False Positive) : False를 True라고 예측한 경우.
- FN(False Negative): True를 False라고 예측한 경우.
- 정확도 = (TP + TN) / (TP + FN + FP + TN)
- 검증 데이터를 활용하여 예상한 정확도, 0.84는 이런 의미를 갖는다.

# Step 6. 제출


In [ ]:
preds=np.round(preds).astype(bool)
sample_submission['Transported'] = preds
sample_submission.to_csv("submission.csv",index=False)
sample_submission.head()

# 점수
- 0.79097 / 730등 / 1187 팀

# Reference
- Spaceship Titanic -EDA + 27 different models
- https://www.kaggle.com/code/odins0n/spaceship-titanic-eda-27-different-models

- Spaceship Titanic: A complete guide
- https://www.kaggle.com/code/samuelcortinhas/spaceship-titanic-a-complete-guide

- 혼동행렬, 분류성능평가지표
- https://data-alpha.tistory.com/22

- 나무위키 : 혼동행렬
- https://namu.wiki/w/%ED%98%BC%EB%8F%99%ED%96%89%EB%A0%AC